# Домашнее задание №1

## Постановка задачи

1. Используется [датасет](https://archive.ics.uci.edu/ml/datasets/Credit+Approval) с данными об одобрении кредитов. Содержит бинарные, категориальные и числовые признаки; также содержит пропущенные данные.
2. Задача - бинарная классификация, целевой признак - последний столбец (A16); показывает - одобрен кредит или нет.
3. Функция потерь - бинарная кроссэнтропия (можно также попробовать Square hinge loss).
4. Метрики оценки качества - accuracy (основная метрика), [ROC AUC score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html), [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

### Бинарная кроссэнтропия

$L(y, \hat{y})=-\frac{1}{N}\sum_{i=0}^{N}(y\cdot \log(\hat{y_{i}})+(1-y)\cdot \log(1-\hat{y_{i}}))$,
$y$ - целевое значение, $\hat{y}$ - предсказанное значение.

### Square hinge loss

$L(y, \hat{y})=-\frac{1}{N}\sum_{i=0}^{N}(max(0,1-y_{i}\cdot\hat{y_{i}})^{2})$

In [1]:
import pandas as pd

## Общая информация о датасете

In [2]:
f = open('crx.names')
for line in f:
    print(line)



1. Title: Credit Approval



2. Sources: 

    (confidential)

    Submitted by quinlan@cs.su.oz.au



3.  Past Usage:



    See Quinlan,

    * "Simplifying decision trees", Int J Man-Machine Studies 27,

      Dec 1987, pp. 221-234.

    * "C4.5: Programs for Machine Learning", Morgan Kaufmann, Oct 1992

  

4.  Relevant Information:



    This file concerns credit card applications.  All attribute names

    and values have been changed to meaningless symbols to protect

    confidentiality of the data.

  

    This dataset is interesting because there is a good mix of

    attributes -- continuous, nominal with small numbers of

    values, and nominal with larger numbers of values.  There

    are also a few missing values.

  

5.  Number of Instances: 690



6.  Number of Attributes: 15 + class attribute



7.  Attribute Information:



    A1:	b, a.

    A2:	continuous.

    A3:	continuous.

    A4:	u, y, l, t.

    A5:	g, p, gg.

    A6:	c, d, cc, i, j, k, m, r, q, w, x, 

In [3]:
df = pd.read_csv('crx.data', sep=',', header=None)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+
